## 트랜스퍼 러닝
- 트랜스퍼 러닝(transfer learning)이란 특정 태스크를 학습한 모델을 다른 태스크 수행에 재사용하는 기법
- 트랜스퍼 러닝이 주목받게 된 것은 업스트림 태스크와 프리트레인 덕분. 자연어의 풍부한 문맥(context)을 모델에 내재화하고 이 모델을 다양한 다운스트림 태스크에 적용해 그 성능을 대폭 끌어올림
- 대표적인 업스트림 태스크 가운데 하나가 다음 단어 맞추기입니다. GPT 계열 모델이 바로 이 태스크로 프리트레인을 수행
- 다른 업스트림 태스크로는 빈칸 채우기가 있습니다. BERT 계열 모델이 바로 이 태스크로 프리트레인을 수행. ‘빈칸 채우기’로 업스트림 태스크를 수행한 모델을 마스크 언어 모델(Masked Language Model)
- 다음 단어 맞히기, 빈칸 채우기 같은 업스트림 태스크는 강력한 힘을 지니며 뉴스, 웹 문서, 백과사전 등 글만 있으면 수작업 없이도 다량의 학습 데이터를 아주 싼값에 만들어 낼 수 있다. 이처럼 데이터 내에서 정답을 만들고 이를 바탕으로 모델을 학습하는 방법을 자기 지도 학습(self-supervised learning)이라고 함.
- 다운스트림 태스크의 학습 방식은 모두 파인튜닝(fine-tuning)이며 프리트레인을 마친 모델을 다운스트림 태스크에 맞게 업데이트하는 기법

## 바이트 페어 인코딩(Byte Pair Encoding, BPE)
- 원래 정보를 압축하는 알고리즘으로 제안되었는데 최근에는 자연어 처리 모델에 널리 쓰이고 있는 토큰화 기법
- 데이터에서 가장 많이 등장한 문자열을 병합해서 데이터를 압축하는 기법

## 워드피스(wordpiece)
- 말뭉치에서 자주 등장한 문자열을 토큰으로 인식한다는 점에서 BPE와 본질적으로 유사. 다만 어휘 집합을 구축할 때 문자열을 병합하는 기준이 다르다. 워드피스는 BPE처럼 단순히 빈도를 기준으로 병합하는 것이 아니라, 병합했을 때 말뭉치의 우도(likelihood)를 가장 높이는 쌍을 병합

In [2]:
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [7]:
# 다양한 언어와 주제의 공개 데이터 세트를 제공하는 파이썬 라이브러리이며 자연어 처리 작업을 위한 데이터를 쉽게 얻을 수 있다.
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 76.9MB/s]                           
[nsmc] download ratings_test.txt: 4.90MB [00:00, 34.1MB/s]                           


In [5]:
import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')
write_lines("/content/train.txt", nsmc.train.get_all_texts()) # 주어진 데이터셋에서 모든 텍스트를 가져오는 함수
write_lines("/content/test.txt", nsmc.test.get_all_texts())

In [8]:
!ls

sample_data  test.txt  train.txt


In [9]:
import os
os.makedirs("/gdrive/My Drive/nlpbook/bbpe", exist_ok=True)

In [10]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"], # 학습 말뭉치를 리스트 형태로 넣기
    vocab_size=10000, # 어휘 집합 크기 조절
    special_tokens=["[PAD]"] # 특수 토큰 추가
)
bytebpe_tokenizer.save_model("/gdrive/My Drive/nlpbook/bbpe")

['/gdrive/My Drive/nlpbook/bbpe/vocab.json',
 '/gdrive/My Drive/nlpbook/bbpe/merges.txt']

In [11]:
# 바이트 레벨 BPE 어휘 집합 구축 결과 (VOCAB.JSON)
!cat /gdrive/MyDrive/nlpbook/bbpe/vocab.json

{"[PAD]":0,"!":1,"\"":2,"#":3,"$":4,"%":5,"&":6,"'":7,"(":8,")":9,"*":10,"+":11,",":12,"-":13,".":14,"/":15,"0":16,"1":17,"2":18,"3":19,"4":20,"5":21,"6":22,"7":23,"8":24,"9":25,":":26,";":27,"<":28,"=":29,">":30,"?":31,"@":32,"A":33,"B":34,"C":35,"D":36,"E":37,"F":38,"G":39,"H":40,"I":41,"J":42,"K":43,"L":44,"M":45,"N":46,"O":47,"P":48,"Q":49,"R":50,"S":51,"T":52,"U":53,"V":54,"W":55,"X":56,"Y":57,"Z":58,"[":59,"\\":60,"]":61,"^":62,"_":63,"`":64,"a":65,"b":66,"c":67,"d":68,"e":69,"f":70,"g":71,"h":72,"i":73,"j":74,"k":75,"l":76,"m":77,"n":78,"o":79,"p":80,"q":81,"r":82,"s":83,"t":84,"u":85,"v":86,"w":87,"x":88,"y":89,"z":90,"{":91,"|":92,"}":93,"~":94,"¡":95,"¢":96,"£":97,"¤":98,"¥":99,"¦":100,"§":101,"¨":102,"©":103,"ª":104,"«":105,"¬":106,"®":107,"¯":108,"°":109,"±":110,"²":111,"³":112,"´":113,"µ":114,"¶":115,"·":116,"¸":117,"¹":118,"º":119,"»":120,"¼":121,"½":122,"¾":123,"¿":124,"À":125,"Á":126,"Â":127,"Ã":128,"Ä":129,"Å":130,"Æ":131,"Ç":132,"È":133,"É":134,"Ê":135,"Ë":136,"Ì":137

In [12]:
# 바이트 레벨 BPE 어휘 집합 구축 결과 (MERGE.TXT)
!cat /gdrive/MyDrive/nlpbook/bbpe/merges.txt

#version: 0.2
Ġ ì
Ġ ë
ì Ŀ
ë ĭ
í ķ
ê °
. .
ìĿ ´
ëĭ ¤
ë Ĭ
ì Ĺ
ê ³
ì §
ëĬ Ķ
ì ŀ
ë §
í Ļ
ê³ ł
ì ł
íĻ Ķ
ĺ ģ
Ġ ê
ë ı
ì ķ
ã ħ
ĺģ íĻĶ
ì ļ
ì§ Ģ
íķ ĺ
ê° Ģ
ë Ĥ
ê ²
ì Ħ
Ġì ŀ
¬ ë
ê ¸
Ġì ķ
ëı Ħ
Ġ í
ë ĵ
ë ¦
ìĹ Ĳ
Ġì Ŀ
íķ ľ
Ġì ĺģíĻĶ
Ī ë
³ ´
ì ĭ
ĸ ´
ìĿ ĺ
ê¸ °
ãħ ĭ
Ġì Ĺ
ìĿ Ģ
ë ¡
ë į
ìĿ Ħ
Ŀ ¼
ëĤ ĺ
ê² Į
ĠìĿ ´
ìĦ ľ
Ġë §
ìļ Ķ
ì Ĭ
ì ĸ´
ë¡ ľ
Ġë Ĥ
ë§ Į
ë Ŀ¼
ë¦ ¬
Ġì ł
· ¸
ëĭ Ī
ëĵ ¤
ë ¥
ê ±
ìķ Ħ
ë ł
.. .
ë ©
¬ ´
ì ľ
Ġ ê°
ìĿ ¸
ãħĭ ãħĭ
¯ ¸
ëį °
Ġì §
ë Ħ
Ġìķ Ħ
Į Ģ
ë Ł
ì ĺ
ê µ
íķ ´
Ġë ³´
ë© ´
ì ĥ
ì ĺģíĻĶ
Ġì ĭ
Ġê ·¸
ê ¹
ë °
Ġë ª
ìł Ĳ
ìĭ ľ
Ī ĺ
Ġë ĭ
ë ³´
ìĿ Į
ìĬ ¤
£ ¼
ë ŀ
Ġë °
ìľ ¼
ëĦ ¤
Ġìŀ ¬ë
ë¥ ¼
ë§ Ĳ
Ġì ¢
! !
ë ¶
ì ¤
Ġì Ĥ
ê± °
ì Ĥ
ì Ļ
ë ĮĢ
ëĭĪ ëĭ¤
Īë ¬´
ìŀ Ĳ
ëĬĶ ëį°
ì Ľ
Ġë ³
ìł ķ
Ġë Ħ
ë§ Ī
ê¹ Į
ì ²
ìĹ Ĩ
ĠìĹ Ĩ
ìĹ Ī
ĠëĤ ĺ
Ġ íķĺ
ìļ °
Ġë ´
ì ¹
ìķ ¼
Ġì¢ ĭ
ì £¼
ì§ Ħ
Ġëĭ ¤
ì Īĺ
í ĸ
ë ³
ë ²
ìł ģ
µ ľ
ìŀ ¥
ìŀ Ī
ìŀ ĳ
ìł Ħ
ìĥ ģ
ë ª
.. ..
Ġì ĥ
Ġìł ķ
ì§ ľ
ì Ĩ
° Į
ìľ¼ ë¡ľ
Ġê ²
Ġìŀ Ī
ì§Ģ ë§Į
í ĺ
ê° Ħ
ĠìĹ °
íķĺ ê³ł
Ġì Ļ
¬ë ŀ
ê³ ¼
Ĳ ëı
ìĺ ¤
Ġì Ĭ
Ġë ĵ
ëĤ ´
Ġê ¸
ı ī
ãħ ł
ĠëĦ Īë¬´
ëŁ °
ë ħ
Ġì ĸ´
Ġì ĺ
Ġë§ Į
í ĥ
Ġìŀ¬ë ¯¸
Ġì§ Ģ
¹ Ħ
ë Ķ
ê ·¸

BERT 토크나이저 구축

In [13]:
import os
os.makedirs("/gdrive/My Drive/nlpbook/wordpiece", exist_ok=True)

In [14]:
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["/content/train.txt", "/content/test.txt"],
    vocab_size=10000,
)
wordpiece_tokenizer.save_model("/gdrive/My Drive/nlpbook/wordpiece")

['/gdrive/My Drive/nlpbook/wordpiece/vocab.txt']

In [15]:
!cat /gdrive/MyDrive/nlpbook/wordpiece/vocab.txt

[PAD]
[UNK]
[CLS]
[SEP]
[MASK]
!
"
%
&
'
(
)
*
+
,
-
.
/
0
1
2
3
4
5
6
7
8
9
:
;
<
=
>
?
@
A
B
C
D
E
F
G
I
K
L
M
N
O
P
R
S
T
V
X
[
]
^
_
`
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
r
s
t
u
v
w
x
y
z
~
★
♡
♥
ㄱ
ㄴ
ㄷ
ㄹ
ㅁ
ㅂ
ㅅ
ㅇ
ㅈ
ㅉ
ㅋ
ㅎ
ㅏ
ㅗ
ㅜ
ㅠ
ㅡ
ㅣ
가
각
간
갈
감
갑
값
갓
갔
강
갖
같
개
객
갠
갱
걍
거
건
걸
검
겁
것
겉
게
겐
겟
겠
겨
격
견
결
겹
겼
경
계
고
곡
곤
골
곱
곳
공
과
관
광
괜
괴
굉
교
구
국
군
굳
굴
굿
궁
권
귀
규
균
그
극
근
글
금
급
기
긴
길
김
깊
까
깎
깐
깔
깜
깝
깨
꺼
껄
껏
께
껴
꼈
꼬
꼭
꼴
꼽
꽃
꽝
꽤
꾸
꾼
꿀
꿈
꿔
뀌
끄
끈
끊
끌
끔
끝
끼
낀
낄
낌
나
낙
낚
난
날
남
납
낫
났
낭
낮
낳
내
낸
낼
냄
냈
냉
냐
냥
너
넌
널
넘
넣
네
넷
녀
년
념
녕
노
녹
논
놀
놈
농
높
놓
놔
놨
뇌
누
눈
뉴
느
는
늘
늙
능
늦
니
닉
닌
닐
님
닙
닝
다
닥
단
닫
달
닮
담
답
당
닿
대
댄
댓
더
덕
던
덜
덤
덩
데
덴
뎁
도
독
돈
돋
돌
동
돼
됐
되
된
될
됨
됩
됬
두
둘
둥
뒤
뒷
드
득
든
듣
들
듬
듭
듯
등
디
딘
딧
딩
따
딱
딴
딸
땅
때
땐
땜
땡
떄
떠
떡
떤
떨
떻
떼
또
똑
똥
뚝
뚱
뛰
뜨
뜩
뜬
뜻
라
락
란
랄
람
랍
랐
랑
래
랙
랜
램
랫
랬
략
량
러
럭
런
럴
럼
럽
렁
렇
레
렉
렌
려
력
련
렬
렵
렷
렸
령
례
로
록
론
롭
롯
롱
뢰
료
룡
루
룬
류
륜
률
륭
르
른
를
름
리
릭
린
릴
림
립
릿
링
마
막
만
많
말
맘
맙
맛
망
맞
맡
매
맥
맨
맹
머
먹
먼
멀
멈
멋
멍
메
멘
멜
며
면
멸
명
몇
모
목
몬
몰
몸
못
몽
묘
무
묵
문
묻
물
뭉
뭐
뭔
뭘
뮤
므
미
믹
민
믿
밀
밋
밌
밑
바
박
밖
반
받
발
밝
밤
밥
방
배
백
뱀
버
번
벌
범
법